# Fugue Interface

In the previous section, we talked about the motivation of Fugue and showed a simple example of Fugue code. Here we provide more details around the Fugue syntax that will be needed in actual applications. We will also talk about some of the underlying concepts that were shown in the previous section.

## The Directed Acyclic Graph (DAG)

In the last section, we had a code block that used `with FugueWorkflow`. 



## Schema

## Passing Parameters

## Decoupling Logic and Execution

To illustrate the first two painpoints above, we'll use a simple code example on a DataFrame with 2 string columns (*col1* and *col2*). This following operation will not make sense, but will show how Pandas and Spark syntactically differ.  We're will create a new column (*col3*). This third column with be a concatenation of the first column with the first 3 letters second column. This whole value will then be reversed. We will execute this logic on Pandas and Spark.